In [1]:
import urllib, http.client
import time
import json
import hmac, hashlib
import pandas as pd

In [2]:
API_KEY ='K-3038939b9a19156293f232b73e6d2392c9799078'
API_SECRET = 'S-c25c5229f8be56020bcd495c8adac52e02ff0c4a'

In [3]:
CURRENCY_1 = 'BTC'
CURRENCY_2 = 'USD'
CURRENCY_1_MIN_QUANTITY = 0.001

In [4]:
ORDER_LIFE_TIME = 3 # минуты
STOCK_FEE = 0.002 # комиссия == 0.2%
AVG_PRICE_PERIOD = 90 # за какой период брать ср цену(минуты)
CAN_SPEND = 5 # сколько тратить CUR_2 при каждой покупке CUR_1
BLOCK_SPEND = 30  # ограничение CUR_2 на покупку
PROFIT_MARKUP = 0.001 # прибыль в 1%
DEBUG = True # инфа отладки
STOCK_TIME_OFFSET = 0 # если время биржи расходится с нашим

# обращение к api
API_URL = 'api.exmo.me'
API_VERSION = 'v1'
CURRENT_PAIR = CURRENCY_1 + '_' + CURRENCY_2

In [5]:
class ExmoAPI:
    def __init__(self, API_KEY, API_SECRET, API_URL = 'api.exmo.me', API_VERSION = 'v1'):
        self.API_URL = API_URL
        self.API_VERSION = API_VERSION
        self.API_KEY = API_KEY
        self.API_SECRET = bytes(API_SECRET, encoding='utf-8')        
    def sha512(self, data):
        H = hmac.new(key = self.API_SECRET, digestmod = hashlib.sha512)
        H.update(data.encode('utf-8'))
        return H.hexdigest()
    def api_query(self, api_method, params = {}):
        params['nonce'] = int(round(time.time() * 1000))
        params =  urllib.parse.urlencode(params)
        sign = self.sha512(params)
        headers = {
            "Content-type": "application/x-www-form-urlencoded",
            "Key": self.API_KEY,
            "Sign": sign
        }   
        conn = http.client.HTTPSConnection(self.API_URL, timeout=60)
        conn.request("POST", "/" + self.API_VERSION + "/" + api_method, params, headers)
        response = conn.getresponse().read()
        conn.close()
        
        try:
            obj = json.loads(response.decode('utf-8'))
            if 'error' in obj and obj['error']:
                print(obj['error'])
                raise sys.exit()
            return obj
        except json.decoder.JSONDecodeError:
            print('Error while parsing response:', response)
            raise sys.exit()

In [6]:
api = ExmoAPI(API_KEY, API_SECRET)
for i in api.api_query('user_info').items():
    print(i)

('uid', 1421646)
('server_date', 1530938219)
('balances', {'USD': '0', 'EUR': '0', 'RUB': '74', 'PLN': '0', 'UAH': '0', 'BTC': '0', 'LTC': '0', 'DOGE': '0', 'DASH': '0', 'ETH': '0', 'WAVES': '0', 'ZEC': '0', 'USDT': '0', 'XMR': '0', 'XRP': '0', 'KICK': '0', 'ETC': '0', 'BCH': '0', 'BTG': '0', 'EOS': '0', 'HBZ': '0', 'BTCZ': '0', 'DXT': '0', 'STQ': '0', 'XLM': '0'})
('reserved', {'USD': '0', 'EUR': '0', 'RUB': '20', 'PLN': '0', 'UAH': '0', 'BTC': '0', 'LTC': '0', 'DOGE': '0', 'DASH': '0', 'ETH': '0', 'WAVES': '0', 'ZEC': '0', 'USDT': '0', 'XMR': '0', 'XRP': '0', 'KICK': '0', 'ETC': '0', 'BCH': '0', 'BTG': '0', 'EOS': '0', 'HBZ': '0', 'BTCZ': '0', 'DXT': '0', 'STQ': '0', 'XLM': '0'})


# Get closed trades 

'amount': '68.97540322', <br>
'date': 'Jun|28|14:11:16|2018',<br>
'price': '6102.30069999',<br>
'quantity': '0.01130318',<br>
'trade_id': 63463481,<br>
'type': 'buy',<br>

In [7]:
large_pac = 10000
small_pac = 1000

In [8]:
def get_trades(l):
    trades = api.api_query('trades',
        {'pair':str(CURRENCY_1)+'_'+str(CURRENCY_2), 'limit':l}).items()
    trades_ = []
    for i in trades:
        trades_.append(i)
        
    trades = trades_[0][1]
    
    '''
    for i in range(0, len(trades)):
        t_d = time.ctime(trades[i].get('date')).split(' ')[1::]
        trades[i]['date'] = '|'.join(t_d)
    '''
    
    return trades[::-1] # traeds по времени 0 - старое, -1 - последнее новое

In [9]:
#метод объединения основного пакета с новым 
def unite_pacs():
    rotate_array = []
    for i in range(1, len(mid_pac)):
        if main_trades[-1].get('date') < mid_pac[-i].get('date'):
            rotate_array.append(mid_pac[-i]) 

    for a in rotate_array[::-1]:
        main_trades.append(a)

    del rotate_array

In [19]:
#получаем основной пакет 10к записей 
main_trades = get_trades(100000) 

In [20]:
#пакет, который обновит  основной при вызве его через интервал и вызове метода unite_pacs()
mid_pac = get_trades(50)

In [21]:
# последняя в главном списке  
print(time.ctime(main_trades[-1].get('date')))

#номер трейда в пакете, который обновляет -> будет добавлено 49 - num в основной пакет
print(mid_pac.index(main_trades[-1]))

Sat Jul  7 14:39:25 2018
49


In [22]:
# добавляем обновленные трейды
unite_pacs()

In [23]:
len(main_trades)

10000

# 30 min info

In [24]:
def create_one_interval(order):
    interval = {}
    interval['total_amount']=float(order['amount'])
    interval['total_quantity']=float(order['quantity'])
    interval['min_buy_price']=None
    interval['min_sell_price']=None
    interval['max_buy_price']=0
    interval['max_sell_price']=0
    interval['total_buy_quantity']=0
    interval['total_sell_quantity']=0
    interval['bought_total_amount']=0
    interval['sell_total_amount']=0
    interval['middle_buy_price'] = 0
    interval['middle_sell_price'] = 0
    interval['middle_price'] = float(order['price'])
    
    if order['type']=='buy':
        interval['max_buy_price']=float(order['price'])
        interval['min_buy_price']=float(order['price'])
        interval['total_buy_quantity']=float(order['quantity'])
        interval['bought_total_amount']=float(order['amount'])
        interval['middle_buy_price'] = float(order['price'])
        
    elif order['type']=='sell':
        interval['max_sell_price']=float(order['price'])
        interval['min_sell_price']=float(order['price'])
        interval['total_sell_quantity']=float(order['quantity'])
        interval['sell_total_amount']=float(order['amount'])
        interval['middle_sell_price'] = float(order['price'])
    return interval
    
def upgrade_one_interval(order, interval):
    #     общее число        
    interval['total_quantity'] += float(order['quantity'])
    interval['total_amount'] += float(order['amount'])
    interval['middle_price'] = interval['total_amount'] / interval['total_quantity']
#     минимальная и максимальная цена покупки
    if (order['type']=='buy'):
        if float(order['price'])>interval['max_buy_price']:
            interval['max_buy_price'] = float(order['price'])
        if interval['min_buy_price'] == None:
            interval['min_buy_price'] = float(order['price'])
        elif (float(order['price'])<interval['min_buy_price']):
            interval['min_buy_price'] = float(order['price'])
#             общ стоимость и объем покупки
        interval['bought_total_amount']+=float(order['amount'])
        interval['total_buy_quantity']+=float(order['quantity'])
        interval['middle_buy_price'] = interval['bought_total_amount']/interval['total_buy_quantity']
#             минимальная и максимальная цена продажи
    elif order['type']=='sell':
        if float(order['price']) > interval['max_sell_price']:
            interval['max_sell_price'] = float(order['price'])
        if interval['min_sell_price'] == None:
            interval['min_sell_price'] = float(order['price'])
        elif float(order['price']) <interval['min_sell_price']:
            interval['min_sell_price'] = float(order['price'])
        interval['sell_total_amount'] += float(order['amount'])
        interval['total_sell_quantity'] += float(order['quantity'])
        interval['middle_sell_price'] = interval['sell_total_amount']/interval['total_sell_quantity']
#     перевес 
    if interval['bought_total_amount']>interval['sell_total_amount']:
        interval['pereves_val'] = interval['bought_total_amount']-interval['sell_total_amount']
#         interval['pereves_ps'] = ((interval['bought_total_amount'] - interval['sell_total_amount'])/ interval['sell_total_amount']) * 100 
        interval['flag'] = 1 #флаг перевеса покупки для классификации
    elif interval['sell_total_amount'] > interval['bought_total_amount']:
        interval['pereves_val'] = interval['sell_total_amount'] - interval['bought_total_amount']
#         interval['pereves_ps'] = ((interval['sell_total_amount'] - interval['bought_total_amount'])/interval['bought_total_amount']) * 100
        interval['flag'] = 0 #флаг перевеса продаж для классификации
    #  немного статистики можнео накидать(тип в % как в выводе функции ниже)
    return interval

In [25]:
trade_time = {}

for order in main_trades[::-1]:
    # если пустой, создаём и назначаем минимальное время
    if trade_time == {}:
        trade_time['%s' % (order['date'])] = create_one_interval(order)
        min_key = order['date']
    #иначе добавляем
    else:
        #ищем минимальный ключ
        for i in trade_time.keys():
            if int(i) < min_key:
                min_key = int(i)
        #если время ордера не входит в промежуток, то создаём новый
        if (min_key - int(order['date'])) > 900:
            trade_time['%s' %(order['date'])] = create_one_interval(order)
        #если нет, то обновляем старый
        else:
            trade_time[str(min_key)] = upgrade_one_interval(order,(trade_time[str(min_key)]))


In [26]:
df = pd.DataFrame.from_dict(trade_time)
df = df.T
print(len(trade_time))

213
